In [39]:
import torch
from nltk.parse import CoreNLPParser

# %cd /home/m25dehgh/simplification/Edit-Unsup-TS
# from normalizing import *


/home/m25dehgh/simplification/Edit-Unsup-TS


In [9]:
# %cd /home/m25dehgh/simplification/asset/dataset
# train_src = open('/home/m25dehgh/simplification/asset/dataset/asset.test.orig', encoding='utf-8').read().split('\n')
# valid_src = open('/home/m25dehgh/simplification/asset/dataset/asset.valid.orig', encoding='utf-8').read().split('\n')
# valid_dst = open('/home/m25dehgh/simplification/asset/dataset/asset.valid.simp.0', encoding='utf-8').read().split('\n')

wiki_orig_out = open('src/Wikilarge/output/RM+EX+LS+RO.txt', encoding='utf-8').read().split('\n')
wiki_me_out = open('src/Wikilarge/output/simplifications_Wikilarge.txt', encoding='utf-8').read().split('\n')



In [25]:
new_wiki = []
for i in range(len(wiki_me_out)):
    if i % 8 == 2:
        new_wiki.append(wiki_me_out[i])

In [27]:
len(new_wiki)

156

In [34]:
len(wiki_orig_out)

with open('src/Wikilarge/output/wiki_me.txt', "a") as file:
    for i in range(len(new_wiki)):
        file.write(new_wiki[i] + "\n")
        

# with open('wiki_me.txt', "r") as file:
#     print(file.read())
#     file.close()

In [33]:
!pwd

/home/m25dehgh/simplification/Edit-Unsup-TS


In [7]:
print(train_src[0])
train_src = [all_norms(i) for i in train_src]
valid_src = [all_norms(i) for i in valid_src]
valid_dst = [all_norms(i) for i in valid_dst]


One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed, a Sudanese militia group recruited mostly from the Afro-Arab Abbala tribes of the northern Rizeigat region in Sudan.


In [3]:

parser = CoreNLPParser(url='http://localhost:9000')
list(parser.parse('What is the airspeed of an unladen swallow hihi hi ?'.split()))
# list(parser.parse(valid_src[233].split()))


[Tree('ROOT', [Tree('SBARQ', [Tree('WHNP', [Tree('WP', ['What'])]), Tree('SQ', [Tree('VBZ', ['is']), Tree('NP', [Tree('NP', [Tree('DT', ['the']), Tree('NN', ['airspeed'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('DT', ['an']), Tree('JJ', ['unladen'])])])])]), Tree('VP', [Tree('VB', ['swallow']), Tree('NP', [Tree('FW', ['hihi'])]), Tree('INTJ', [Tree('UH', ['hi'])])]), Tree('.', ['?'])])])]

In [16]:
valid_src[232].split()

['the',
 'state',
 'covers',
 'an',
 'area',
 'of',
 '74,122',
 'sq',
 'mi',
 '-LRB-',
 '191,976',
 'km',
 '²',
 '-RRB-',
 ',',
 'or',
 '5.83',
 '%',
 'of',
 'the',
 'total',
 'geographical',
 'area',
 'of',
 'india',
 '.']

In [ ]:
"one side of the armed conflicts is composed mainly of the sudanese military and the janjaweed , a sudanese militia group recruited mostly from the afro - arab abbala tribes of the northern rizeigat region in sudan ."
"one side of the armed disputes is produced mainly of the sudanese military and the janjaweed , a sudanese rebel group ."
"one side of the armed disputes is composed mainly of the sudanese military and the janjaweed , a sudanese rebel group recruited mostly from the afro - arab abbala tribes of the northern rizeigat region in sudan ."

"jeddah is the principal gateway to mecca , islam ` s holiest city , which able - bodied muslims are required to visit at least once in their lifetime ."
"jeddah is the principal gateway to mecca , islam 's holiest city , which able-bodied muslims are required to visit at least once in their lifetime ."
"jeddah is the principal gateway to mecca , islam ` s able - stout holiest city , which muslims are needed to leave at least once in their lifetime ."
